In [1]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")

config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/feature_platform")
alpha_list = list(FT.features_in_path.keys())[:5]

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [4]:
start_date = '2020-01-01'
end_date = '2021-06-01'
target = "10d_ret_StandardScale_MinMax_10"

In [17]:
def generate_original_data(alpha_name,alpha_list,target,start_date,end_date):
    
    dataloader = DataLoader(None)
    dataloader.load_data_from_file(data_path="/home/wuwenjun/feature_platform/ti0/wuwenjun/%s/" % (target) ,start_date=start_date,end_date=end_date)
    dataloader.feature_data.rename(columns= {target:"target"},inplace=True)
    FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None)
    output = pd.concat([FT.feature_data,dataloader.feature_data],axis=1)
    convert_to_standard_daily_data_par(df=output,output_name=alpha_name,output_path="/home/wuwenjun/Data_lib/ti0/wuwenjun/")
    return output

In [19]:
a = generate_original_data(alpha_name="test",alpha_list=alpha_list,target=target,start_date=start_date,end_date=end_date)
a

Loading Feature...: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1700/1700 [00:03<00:00, 501.52it/s]


test_mcorr_open_low_10    test_1     test  \
timestamp           ticker                                              
2020-01-02 09:30:00 000001                0.115025  0.115025  3604.78   
                    000002                0.796874  0.796874  9428.61   
                    000004                0.815852  0.815852   184.01   
                    000005                0.815519  0.815519    57.28   
                    000006                0.846951  0.846951   379.05   
...                                            ...       ...      ...   
2021-06-01 09:30:00 688699                     NaN       NaN      NaN   
                    688777                     NaN       NaN      NaN   
                    688788                     NaN       NaN      NaN   
                    688819                     NaN       NaN      NaN   
                    688981                     NaN       NaN      NaN   

                            ewmCorr(high,low)  ewmCorr(high,low,7)    target  
timestamp           ticker                                                    
2020-01-02 09:30:00 000001           0.414245             0.414245 -5.551395  
                    000002           0.968666             0.968666 -6.203258  
                    000004          -0.086929            -0.086929 -5.390784  
                    000005           0.656316             0.656316 -5.142255  
                    000006           0.967827             0.967827 -5.327572  
...                                       ...                  ...       ...  
2021-06-01 09:30:00 688699                NaN                  NaN -6.236975  
                    688777                NaN                  NaN -6.432447  
                    688788                NaN                  NaN -6.486812  
                    688819                NaN                  NaN -6.124710  
                    688981                NaN                  NaN -6.088935  

[1366034 rows x 6 columns]

In [21]:
from AlphaNet.Data import DataLoader
dataloader = DataLoader("None")
dataloader.load_data_from_file(data_path="/home/wuwenjun/Data_lib/ti0/wuwenjun/test/",end_date="2022-01-01")

test_mcorr_open_low_10    test_1     test  \
timestamp           ticker                                              
2020-01-02 09:30:00 000001                0.115025  0.115025  3604.78   
                    000002                0.796874  0.796874  9428.61   
                    000004                0.815852  0.815852   184.01   
                    000005                0.815519  0.815519    57.28   
                    000006                0.846951  0.846951   379.05   
...                                            ...       ...      ...   
2021-06-01 09:30:00 688699                     NaN       NaN      NaN   
                    688777                     NaN       NaN      NaN   
                    688788                     NaN       NaN      NaN   
                    688819                     NaN       NaN      NaN   
                    688981                     NaN       NaN      NaN   

                            ewmCorr(high,low)  ewmCorr(high,low,7)    target  
timestamp           ticker                                                    
2020-01-02 09:30:00 000001           0.414245             0.414245 -5.551395  
                    000002           0.968666             0.968666 -6.203258  
                    000004          -0.086929            -0.086929 -5.390784  
                    000005           0.656316             0.656316 -5.142255  
                    000006           0.967827             0.967827 -5.327572  
...                                       ...                  ...       ...  
2021-06-01 09:30:00 688699                NaN                  NaN -6.236975  
                    688777                NaN                  NaN -6.432447  
                    688788                NaN                  NaN -6.486812  
                    688819                NaN                  NaN -6.124710  
                    688981                NaN                  NaN -6.088935  

[1366034 rows x 6 columns]